In [1]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

In [2]:
json_file_path = './cv_train_1.json'

In [3]:
with open(json_file_path, 'r') as j :
    json_data = json.load(j)

In [4]:
print(json_data['categories'])

[{'id': 0, 'name': 'General trash', 'supercategory': 'General trash'}, {'id': 1, 'name': 'Paper', 'supercategory': 'Paper'}, {'id': 2, 'name': 'Paper pack', 'supercategory': 'Paper pack'}, {'id': 3, 'name': 'Metal', 'supercategory': 'Metal'}, {'id': 4, 'name': 'Glass', 'supercategory': 'Glass'}, {'id': 5, 'name': 'Plastic', 'supercategory': 'Plastic'}, {'id': 6, 'name': 'Styrofoam', 'supercategory': 'Styrofoam'}, {'id': 7, 'name': 'Plastic bag', 'supercategory': 'Plastic bag'}, {'id': 8, 'name': 'Battery', 'supercategory': 'Battery'}, {'id': 9, 'name': 'Clothing', 'supercategory': 'Clothing'}]


In [23]:
df_annotations = json_normalize(json_data['annotations'])

In [24]:
df_annotations_0 = df_annotations[df_annotations['category_id'] == 0]

In [25]:
df_annotations_0

,image_id,category_id,area,bbox,iscrowd,id
0,0,0,257301.66,"[197.6, 193.7, 547.8, 469.7]",0,0
6,1,0,38908.72,"[425.3, 681.9, 216.4, 179.8]",0,6
8,1,0,6857.76,"[622.4, 686.5, 72.8, 94.2]",0,8
13,5,0,12162.85,"[320.0, 233.7, 182.9, 66.5]",0,19
14,5,0,10264.35,"[297.7, 212.6, 166.9, 61.5]",0,20
...,...,...,...,...,...,...
18629,4875,0,9353.55,"[500.1, 596.8, 147.3, 63.5]",0,23121
18632,4876,0,9096.78,"[199.6, 552.3, 107.4, 84.7]",0,23124
18633,4876,0,52745.00,"[290.8, 722.3, 175.0, 301.4]",0,23125
18637,4880,0,367397.36,"[193.3, 209.2, 576.4, 637.4]",0,23134


In [26]:
bbox_count_class0 = df_annotations_0['image_id'].value_counts().value_counts()
bbox_count_class0

1     1054
2      303
3      145
5       53
4       52
6       26
7       18
8       11
9        7
10       6
12       2
19       1
11       1
13       1
14       1
15       1
16       1
28       1
Name: image_id, dtype: int64

In [27]:
df_annotations_1 = df_annotations[df_annotations['category_id'] == 1]

In [28]:
bbox_count_class1 = df_annotations_1['image_id'].value_counts().value_counts()
bbox_count_class1

1     617
2     206
3     121
4      76
5      75
6      48
7      43
8      30
9      24
11     19
12     18
14     14
10     14
16     10
13      9
15      6
23      6
17      5
18      5
21      5
25      5
19      4
22      3
24      2
34      1
20      1
27      1
31      1
33      1
38      1
Name: image_id, dtype: int64

In [9]:
bbox_count = df_annotations['image_id'].value_counts().value_counts()
print(bbox_count.sum())
under40 = bbox_count[bbox_count.index < 40].sum()
print(under40)
over40 = bbox_count[bbox_count.index >= 40].sum()
print(over40)

3908
3895
13


In [10]:
train_box_count = [0] * 5
train_box_count_under = [0] * 5
train_box_count_over = [0] * 5
train_cnt_under = [0] * 5
val_box_count = [0] * 5
val_box_count_under = [0] * 5
val_box_count_over = [0] * 5
val_cnt_under = [0] * 5

In [11]:
for i in range(1,6):
    json_file_path = f'./cv_train_{i}.json'
    with open(json_file_path, 'r') as j :
        json_data = json.load(j)
    df_annotations = json_normalize(json_data['annotations'])
    bbox_count = df_annotations['image_id'].value_counts().value_counts()
    train_box_count[i-1] = bbox_count.sum()
    train_box_count_under[i-1] = bbox_count[bbox_count.index < 40].sum()
    train_box_count_over[i-1] = bbox_count[bbox_count.index >= 40].sum()
    
    for j in range(len(df_annotations)):
        if df_annotations['bbox'][j][2] < 10 or df_annotations['bbox'][j][3] < 10:
            train_cnt_under[i-1] += 1
            
    json_file_path = f'./cv_val_{i}.json'
    with open(json_file_path, 'r') as j :
        json_data = json.load(j)
    df_annotations = json_normalize(json_data['annotations'])
    bbox_count = df_annotations['image_id'].value_counts().value_counts()
    val_box_count[i-1] = bbox_count.sum()
    val_box_count_under[i-1] = bbox_count[bbox_count.index < 40].sum()
    val_box_count_over[i-1] = bbox_count[bbox_count.index >= 40].sum()
    
    for j in range(len(df_annotations)):
        if df_annotations['bbox'][j][2] < 10 or df_annotations['bbox'][j][3] < 10:
            val_cnt_under[i-1] += 1

In [12]:
print("train 데이터셋")
print("전체 이미지 개수: ", train_box_count)
print("bbox 40개 미만 : ", train_box_count_under)
print("bbox 40개 이상 : ", train_box_count_over)
print("가로든 세로든 10 미만인 박스의 갯수 : ", train_cnt_under)
print("------------------------------")
print("val 데이터셋")
print("전체 이미지 개수: ", val_box_count)
print("bbox 40개 미만 : ", val_box_count_under)
print("bbox 40개 이상 : ", val_box_count_over)
print("가로든 세로든 10 미만인 박스의 갯수 : ", val_cnt_under)

train 데이터셋
전체 이미지 개수:  [3908, 3928, 3911, 3878, 3907]
bbox 40개 미만 :  [3895, 3917, 3902, 3866, 3896]
bbox 40개 이상 :  [13, 11, 9, 12, 11]
가로든 세로든 10 미만인 박스의 갯수 :  [23, 29, 28, 29, 27]
------------------------------
val 데이터셋
전체 이미지 개수:  [975, 955, 972, 1005, 976]
bbox 40개 미만 :  [974, 952, 967, 1003, 973]
bbox 40개 이상 :  [1, 3, 5, 2, 3]
가로든 세로든 10 미만인 박스의 갯수 :  [11, 5, 6, 5, 7]
